# Low-Rank Matrix Completion (LRMC)

In [1]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def svt(mat, tau):
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    vec = s - tau
    vec[vec < 0] = 0
    return u @ np.diag(vec) @ v

def LRMC(dense_mat, sparse_mat, rho, maxiter):
    
    pos_train = np.where(sparse_mat != 0)
    mat_train = sparse_mat[pos_train]
    pos_test = np.where((sparse_mat == 0) & (dense_mat != 0))
    mat_test = dense_mat[pos_test]
    
    X = sparse_mat.copy()
    Z = sparse_mat.copy()
    T = sparse_mat.copy()
    del dense_mat, sparse_mat
    show_iter = 1
    for it in range(maxiter):
        Z = svt(X + T / rho, 1 / rho)
        X = Z - T / rho
        X[pos_train] = mat_train
        T = T - rho * (Z - X)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(mat_test, X[pos_test]))
            print(compute_rmse(mat_test, X[pos_test]))
            print()
    return X

In [2]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.9
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
rho = 1e-4
maxiter = 50
X = LRMC(dense_mat, sparse_mat, rho, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

1
0.7392966368849136
55.59912491627884

2
0.4026454811819589
30.312193535360656

3
0.21320659779890574
15.084731560388459

4
0.2405833699440661
14.903160643205432

5
0.41820783371003906
24.49517875604753

6
0.5398215784326753
31.911487142659297

7
0.5970294736558385
35.573670287501464

8
0.5905986417769161
35.437052001420774

9
0.5276676993997892
31.900082483399608

10
0.4207209893050117
25.69892610997643

11
0.28674401050773946
17.845100150551495

12
0.148833315843925
9.799194545664824

13
0.07797445214001819
5.889277794946034

14
0.1602950979579074
10.495696338497812

15
0.2527807505074229
15.94764517189872

16
0.3140262620800643
19.71858875416167

17
0.33908470257294254
21.345782243856444

18
0.3289799435587809
20.835658046204653

19
0.28893612127949203
18.46511386444483

20
0.2273441604598719
14.717798872143286

21
0.15493721285010137
10.279700783757468

22
0.08861145604649083
6.2870066974751575

23
0.07169381211802421
5.237807836966766

24
0.11690215789834393
7.586816392349698

25

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>